In [2]:
import os
os.getcwd()

'/home/vmadmin/goqba/mk2'

In [5]:
# 파일 자연정렬 라이브러리
!pip install natsort

In [7]:
# YOLO 좌표 변환 라이브러리
!pip install pybboxes

# 프레임으로 분할 후 각 지정 폴더에 저장

In [4]:
import os
import cv2
import natsort

def split_videos_in_folder(path):

            # 바운딩 영상을 가져올 폴더
            input_folder = path + '/video/'

            # 프레임을 저장할 폴더
            output_folder = path + '/frame/'

            # 입력 폴더의 모든 파일을 가져옵니다. 지 마음대로 가져옴
            # files = os.listdir(input_folder)

            # 파일 정렬
            files = natsort.natsorted(os.listdir(input_folder))

            # 프레임을 저장할 폴더를 생성합니다.
            os.makedirs(output_folder, exist_ok=True)

            for file in files:
                frame_count = 0
                
                # 파일의 확장자를 가져옵니다.
                _, ext = os.path.splitext(file)

                # 파일이 영상 파일인지 확인합니다. (확장자가 .mp4, .avi 등)
                if ext.lower() in [".mp4", ".avi"]:
                    # 영상 파일을 엽니다.
                    video_path = os.path.join(input_folder, file)
                    cap = cv2.VideoCapture(video_path)

                    # 비디오 열기를 실패하면 다음 파일로 넘어갑니다.
                    if not cap.isOpened():
                        print(f"{file} 비디오 열기 실패")
                        continue

                    # 동영상 파일명에서 확장자 제거하여 저장할 이미지 파일명으로 사용합니다.
                    video_name = os.path.splitext(file)[0]

                    while True:
                        # 프레임을 읽습니다.
                        ret, frame = cap.read()
                        if not ret:
                            break

                        # 프레임을 이미지 파일로 저장합니다.
                        frame_path = os.path.join(output_folder, f"{video_name}_{frame_count}.jpg")
                        cv2.imwrite(frame_path, frame)

                        frame_count += 1

                    cap.release()
                    print(f"{file} 프레임 분할 완료")

# 기본 경로
path = "/home/vmadmin/goqba/mk2/extractTest"

def main():
    split_videos_in_folder(path)
    
if __name__ == "__main__":
    main()
    cv2.destroyAllWindows()

29254-1a.mp4 프레임 분할 완료
29254-2a.mp4 프레임 분할 완료
29254-3a.mp4 프레임 분할 완료
29254-4a.mp4 프레임 분할 완료
29254-5a.mp4 프레임 분할 완료
41377-1a.mp4 프레임 분할 완료
41377-2a.mp4 프레임 분할 완료
41377-3a.mp4 프레임 분할 완료
41377-4a.mp4 프레임 분할 완료
41377-5a.mp4 프레임 분할 완료
44808-1a.mp4 프레임 분할 완료
44808-2a.mp4 프레임 분할 완료
44808-3a.mp4 프레임 분할 완료
44808-4a.mp4 프레임 분할 완료
44808-5a.mp4 프레임 분할 완료
60061-1a.mp4 프레임 분할 완료
60061-2a.mp4 프레임 분할 완료
60061-3a.mp4 프레임 분할 완료
60061-4a.mp4 프레임 분할 완료
60061-5a.mp4 프레임 분할 완료
98645-3a.mp4 프레임 분할 완료


# 3가지 이미지 데이터셋에 바운딩 박스 적용 및 텍스트 파일 생성
# 임계값 설정 파트 코드 55~71

### 흰색 객체만 남기고 나머지는 전부 검은색 처리 코드 추가 부분

### 동작 과정: 
### 1. 프레임 이미지와 동일한 검은색 배경 이미지를 생성함
### 2. findContours를 통해 얻어낸 바운딩 박스 좌푯값 즉 흰색 객체를 검은색 배경 이미지에 붙여넣기
### 3. 수정된 이미지를 저장함

#### 코드 위치: 53 92 134 

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pybboxes as pbx
import natsort

def find_white_object(path):
    # 프레임을 가져올 폴더
    input_folder = path + '/frame/'

    # 프레임(바운딩박스가 적용된)을 저장할 폴더
    output_folder = path + '/bound/'
        
    # 프레임(흰색 객체만 남도록 적용된)을 저장할 폴더
    output_folder2 = path + '/extraction/'

    # 프레임 내 좌푯값을 저장할 텍스트의 폴더 경로
    txt_folder = path + '/loc/'

    # 입력 폴더의 모든 파일을 가져옴
    files = natsort.natsorted(os.listdir(input_folder))

    # 바운딩 박스 프레임을 저장할 폴더 생성
    os.makedirs(output_folder, exist_ok=True)
        
    # 흰색 객체 추출 프레임을 저장할 폴더 생성
    os.makedirs(output_folder2, exist_ok=True)

    # 텍스트를 저장할 폴더 생성
    os.makedirs(txt_folder, exist_ok=True)

    # 바운딩 박스들의 라벨 초기화
    box_labels = []
    
    frame_count = 0
    
    for file in files:

        # 파일의 확장자를 가져옵니다.
        _, ext = os.path.splitext(file)

        # 파일이 이미지 파일인지 확인합니다. (확장자가 .jpg, .jpeg, png 등)
        if ext.lower() in [".jpg", ".jpeg", "png"]:
        
            # 이미지 파일 경로 결합
            image_path = os.path.join(input_folder, file)

            # 이미지 로드
            image = cv2.imread(image_path)
            
            # 바운딩 박스로 흰색 객체만 있는 이미지 생성
            white_object_image = np.zeros_like(image)  # 원본 이미지 크기와 동일한 검은색 배경 생성

            # 이미지를 그레이스케일로 변환
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # 이미지 명암 계산
            # 이미지를 BGR에서 RGB로 변환함: 명암 계산 용도로만 사용함
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 이미지를 평균화하여 어두운 정도 측정
            darkness_ratio = np.mean(image_rgb) / 255.0

            # threshold의 임계값 설정: 2번째 속성값
            # 해당 위치 픽셀값이 임계값을 넘어선 경우 흰색으로 처리하며 넘기지 못하면 검은색으로 인식함
            Contrast = 0
            if ((darkness_ratio >= 0.5) and (darkness_ratio <= 1)):
                Contrast = 230
            elif ((darkness_ratio > 0) and (darkness_ratio < 0.3)):
                Contrast = 90
            else:
                Contrast = 190

            # 이미지 이진화 (흰색 객체 검출을 위해)
            _, binary_image = cv2.threshold(gray_image, Contrast, 255, cv2.THRESH_BINARY)

            # 이진화된 이미지를 사용하여 흰색 바운딩 박스 검출
            contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
            # 흰색 객체 이미지 파일명 생성 (확장자 제외)
            object_filename = os.path.splitext(file)[0]

            # 텍스트 파일을 쓰기 모드로 열기
            with open(f"{txt_folder}{object_filename}.txt", "w") as file_txt:

                # 각 박스에 고유 번호 부여하고 좌표 구하기
                for idx, contour in enumerate(contours):
                    x, y, w, h = cv2.boundingRect(contour)

                    # 흰색 객체 부분만 해당 이미지에 복사
                    white_object_image[y:y+h, x:x+w] = image[y:y+h, x:x+w]
                    
                    # 바운딩 박스에 라벨값 0 적용
                    box_labels.append(0)

                    # 박스 크기 조정 (2배 확대)
                    w *= 3
                    h *= 3
                    x -= w // 2
                    y -= h // 2

                    # 바운딩 박스의 크기가 20X20라면 해당 박스를 표시하지 않음
                    if ((w >= 20) and (h >= 20)):
                        continue

                    # 흰색 바운딩 박스 그리기
                    # grayscale이 안된 원본 이미지(image)에 바운딩 박스 처리를 수행하고 있음
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 1)

                    # 바운딩 박스 위 고유 번호 텍스트 크기 조정 및 출력 (절반으로 줄임)
                    #font_scale = 0.3
                    #font_thickness = 1
                    #font = cv2.FONT_HERSHEY_SIMPLEX
                    #text_size = cv2.getTextSize(str(idx), font, font_scale, font_thickness)[0]
                    #text_x = x + (w - text_size[0]) // 2
                    #text_y = y - 5
                    # cv2.putText(image, str(idx), (text_x, text_y), font, font_scale, (0, 255, 0), font_thickness)

                    # 바운딩 박스의 꼭짓점 값 4개 구하기
                    box_points = [(x, y), (x + w, y), (x + w, y + h), (x, y + h)]
                    file_txt.write(f"{box_labels[idx]} {box_points[0]} {box_points[1]} {box_points[2]} {box_points[3]}\n")

                    # YOLO 좌표 변환 메서드 호출
                    voc_bbox = (x, y, (x + w), (y + h))
                    W, H = 640, 480  # WxH of the image
                    yolo_LOC = pbx.convert_bbox(voc_bbox, from_type="voc", to_type="yolo", image_size=(W,H))

                    # 바운딩 박스 정보를 텍스트 파일에 저장
                    file_txt.write(f"{box_labels[idx]} {yolo_LOC[0]} {yolo_LOC[1]} {yolo_LOC[2]} {yolo_LOC[3]}\n")

                        
            # 검출한 흰색 객체만 있는 이미지를 저장(수정해야됨[안됨])
            cv2.imwrite(f"{output_folder2}{object_filename}.jpg", white_object_image)

            # 원본 이미지에 바운딩 박스 표시한 이미지 저장
            cv2.imwrite(f"{output_folder}{object_filename}_frameBound.jpg", image)
            frame_count = frame_count + 1

    return box_labels

# 기본 경로
path = "/home/vmadmin/goqba/mk2/extractTest"

def main():    
    # 흰색 바운딩 박스 검출 및 표시
    find_white_object(path)
    
if __name__=="__main__":
    main()
    cv2.destroyAllWindows()
